In [ ]:
import cv2
import os
import datetime
import numpy as np
from PIL import Image
import pymysql as sql

class smart_Attendance:
    print("\t\t\t\t\t\t Smart_Attendance        ")

    def home(self):

        print("""               
            1. Id create 
            2. Attendance
            3. Check Attendance Date
            4. Total Attendance 
            5. Details Check
            6. Exit 
            """)

        print()

        cho = int(input("Enter the Choice : "))
        if cho == 1:
            print()
            self.create()

        elif cho == 2:
            print()
            self.attendance()
        elif cho == 3:
            self.check_attendance_date()
        elif cho == 4:
            self.total_attendance()
        elif cho == 5:
            self.detail_check()

        else:
            exit()

    def create(self):
        face_cascade = cv2.CascadeClassifier("C:/Users/Rames/anaconda file data/Machine learning Sir/xml file/haarcascade_frontalface_default.xml")
        
        recognizer = cv2.face.LBPHFaceRecognizer_create()
        path = "I:/Code Run/Anaconda/Atten"

        cam = cv2.VideoCapture(0)
        Id = int(input("Enter you Id No : "))
        name = input("Enter the Name : ")
        s = datetime.datetime.now()
        t = s.date()
        date = t.strftime('%d/%m/%G')
        at = "P"

        db = sql.connect(host='localhost', user='root', passwd='12345', db='attendance')
        cur = db.cursor()
        cur.execute("select *from s_a")

        c=""
        for i in cur:
            for j in i:
                if j == Id:
                    c = j
                    break

        if c == "":
            db = sql.connect(host='localhost', user='root', passwd='12345', db='attendance')
            cur = db.cursor()
            cur.execute("insert into s_a values(%s,%s,%s,%s)", (Id, name, date, at))
            db.commit()
            db.close()

            sampleNum = 0

            while True:
                ret, img = cam.read()
                faces = face_cascade.detectMultiScale(img, 1.3, 5)
                for (x, y, w, h) in faces:
                    cv2.rectangle(img, (x, y), (x + w, y + h), (144, 1, 1), 3)
                    sampleNum = sampleNum + 1
                    id = str(Id)
                    cv2.imwrite("I:/Code Run/Anaconda/Atten/." + id + "." + str(sampleNum) + ".jpg",img[y:y + h, x:x + w])
                    cv2.imshow("frame", img)
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
                elif sampleNum > 10:
                    break

            cam.release()
            cv2.destroyAllWindows()

            recognizer = cv2.face.LBPHFaceRecognizer_create()
            path = "I:/Code Run/Anaconda/Atten"

            def getImagesWithID(path):
                imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
                faces = []
                IDs = []
                for imagePath in imagePaths:
                    faceImg = Image.open(imagePath).convert('L')
                    faceNp = np.array(faceImg, 'uint8')
                    ID = int(os.path.split(imagePath)[-1].split('.')[1])
                    faces.append(faceNp)
                    IDs.append(ID)
                    cv2.imshow("training", faceNp)
                    cv2.waitKey(100)  # for 10 milliseconds wait......
                return np.array(IDs), faces

            Ids, faces = getImagesWithID(path)
            recognizer.train(faces, np.array(Ids))
            recognizer.save("C:/Users/Rames/Machine learning Sir/xml file/trainningData.yml")
            cv2.destroyAllWindows()

            print()
            print("Successful")

        else:
            print("Already Create")


        print()
        print("1.Home")
        print("2.Id create ")
        print("3.Exit")
        print("")
        a = int(input("Enter the Choice : "))
        if a == 1:
            self.home()
        elif a == 2:
            self.create()
        else:
            exit()

    def attendance(self):
        s = datetime.datetime.now()
        t = s.date()
        date = t.strftime('%d/%m/%G')
        date = "10/5/223"


        faceDetect = cv2.CascadeClassifier("C:/Users/Rames/Machine learning Sir/xml file/haarcascade_frontalface_default.xml")
        cam = cv2.VideoCapture(0)
        rec = cv2.face.LBPHFaceRecognizer_create()
        rec.read("C:/Users/Rames/Machine learning Sir/xml file/trainningData.yml")


        while True:
            ret, img = cam.read()
            gray = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
            faces = faceDetect.detectMultiScale(gray, 2, 4)
            for (x, y, w, h) in faces:
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
                id, conf = rec.predict(gray[y:y + h, x:x + w])

                db = sql.connect(host='localhost', user='root', passwd='12345', db='attendance')
                cur = db.cursor()
                cur.execute("select *from s_a")

                ct = ""
                for i in cur:
                    for j in i:
                        if j == id:
                            ct = j
                            break

                print(ct)

                if ct!= "":

                    db = sql.connect(host='localhost', user='root', passwd='12345', db='attendance')
                    cur = db.cursor()
                    cur.execute(("select *from s_a where ID='%s'") % (id))

                    c = ""
                    for i in cur:
                        for j in i:
                            if j == date:
                                c = j
                                break

                    if c == "":
                        db = sql.connect(host='localhost', user='root', passwd='12345', db='attendance')
                        cur = db.cursor()
                        cur.execute(("select *from s_a where ID='%s'") % (id))
                        store = cur.fetchone()
                        ate = "P"
                        datae = []
                        for i in store:
                            datae.append(i)
                        cur.execute("insert into s_a values(%s,%s,%s,%s)", (datae[0], datae[1], date, ate))
                        db.commit()
                        db.close()
                        print("Successful")
                        #cv2.putText((img, str(datae[1]),(x,y+h),font,3,255))
                        break

                    else:
                        #cv2.putText((img, str(datae[1]), (x, y + h), font, 3, 255))
                        print("Present")

                else:
                    print("Face not Match")




            cv2.imshow("face", img)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        cam.release()
        cv2.destroyAllWindows()

        print()
        print("1.Home")
        print("2.Id create ")
        print("3.Exit")
        print("")
        a = int(input("Enter the Choice : "))
        if a == 1:
            self.home()
        elif a == 2:
            self.create()
        else:
            exit()

    def check_attendance_date(self):

        idea = int(input("Enter The ID : "))
        datee = input("Enter The Date : ")

        db = sql.connect(host='localhost', user='root', passwd='12345', db='attendance')
        cur = db.cursor()
        cur.execute(("select *from s_a where ID='%s'") % (idea))
        x = ""
        for i in cur:
            for j in i:
                if j == datee:
                    x = j
                    break

        if x == datee:
            print("Present")
        else:
            print("apsent")


        print()
        print("1.Home")
        print("2.Id create ")
        print("3.Exit")
        print("")
        a = int(input("Enter the Choice : "))
        if a == 1:
            self.home()
        elif a == 2:
            self.create()
        else:
            exit()

    def total_attendance(self):
        idea = int(input("Enter The ID : "))

        db = sql.connect(host='localhost', user='root', passwd='12345', db='attendance')
        cur = db.cursor()
        cur.execute(("select Date from s_a where ID='%s'") % (idea))
        c=0
        for i in cur:
            for j in i:
                c=c+1

        print(c)

        print()
        print("1.Home")
        print("2.Id create ")
        print("3.Exit")
        print("")
        a = int(input("Enter the Choice : "))
        if a == 1:
            self.home()
        elif a == 2:
            self.create()
        else:
            exit()

    def detail_check(self):

        try:
            idea = int(input("Enter The ID : "))

            db = sql.connect(host='localhost', user='root', passwd='12345', db='attendance')
            cur = db.cursor()
            cur.execute(("select ID,Name from s_a where ID='%s'") % (idea))
            store = cur.fetchone()
            d = []
            for i in store:
                d.append(i)
            print("ID :-", d[0])
            print("Name : ", d[1])
        except Exception as es:
            print(es)

        print()
        print("1.Home")
        print("2.Id create ")
        print("3.Exit")
        print("")
        a = int(input("Enter the Choice : "))
        if a == 1:
            self.home()
        elif a == 2:
            self.create()
        else:
            exit()



object = smart_Attendance()
object.home()
object.create()
object.attendance()
object.total_attendance()
object.detail_check()